In [ ]:
# Essential imports for data analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical summaries
from tableone import TableOne

# BigQuery interaction
import pandas_gbq as pgbq
from google.cloud import bigquery

# Database connections
from sqlalchemy import create_engine

# Utilities for date/time manipulation
from datetime import datetime, timedelta

# Set up inline plotting for Jupyter Notebook
%matplotlib inline


In [ ]:
# Define configurations for Big Query
project_id = '' # Location of stride datalake
client = bigquery.Client(project=project_id) # Set project to project_id
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''
os.environ['GCLOUD_PROJECT'] = "" # specify environment
db = "" # Define the database
stanford_ds = ""
yh_ds = ""

In [ ]:
def save_table(project_id, yh_ds, new_table_name, query):
    table_id = f"{project_id}.{yh_ds}.{new_table_name}"
    job_config = bigquery.QueryJobConfig(destination=table_id)
    job_config.write_disposition = "WRITE_TRUNCATE"
    # Start the query, passing in the extra configuration.
    query_job = client.query(query, job_config=job_config)  # Make an API request.
    query_job.result()  # Wait for the job to complete.
    print("Query results loaded to the table {}".format(table_id))  
def load_pgbq(project_id, yh_ds, table_name):
    sql_query = f"SELECT * FROM {project_id}.{yh_ds}.{table_name}"
    return_df = pgbq.read_gbq(sql_query, dialect="standard")
    print (f"{project_id}.{yh_ds}.{table_name}", "is loaded") 
    return return_df
def upload_pgbq(project_id, yh_ds, table_name, df):
    table_id = f"{yh_ds}.{table_name}"
    pgbq.to_gbq(df, table_id, project_id=project_id)
    print ("dataframe", df, "is uploaded as", f"{project_id}.{yh_ds}.{table_name}") 
def remove_table(project_id, yh_ds, table_name):
    client = bigquery.Client()
    table_id = f"{project_id}.{yh_ds}.{table_name}"
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

In [ ]:
def get_death_duration_column(death_after_exposure,  observation_days): 
    if pd.isna(death_after_exposure):
        return observation_days
    elif death_after_exposure > observation_days: # when the patient died after the observation cut off 
        return observation_days 
    elif death_after_exposure <= 0:
        return 0
    else:
        return death_after_exposure 
def get_death_outcome_column(death_after_exposure, observation_days): 
    if pd.isna(death_after_exposure):
        return False
    elif death_after_exposure > observation_days: 
        return False
    else:
        return True

In [ ]:
cohort_df = pd.read_csv("../A_Cohort/final_cohort_07312024.csv", low_memory = False)

In [ ]:
df_comparison = cohort_df[cohort_df.exposure_group.isin(['new user', 'consistent user'])]

In [ ]:
# Mapping 'deceased' column to boolean values
death_mapping_dict = {'Y': True, 'N': False}
df_comparison = df_comparison.assign(
    deceased_boolean=df_comparison['deceased'].map(death_mapping_dict)
)

# Mapping 'exposure_group' column to numeric values
exposure_mapping_dict = {'new user': 1, 'consistent user': 0}
df_comparison = df_comparison.assign(
    exposure_float=df_comparison['exposure_group'].map(exposure_mapping_dict)
)

# Display the count of each category in the 'exposure_group' column
exposure_group_counts = df_comparison['exposure_group'].value_counts()
print(exposure_group_counts)


In [ ]:
medication_table_name = 'dementia_medication_categories'
medication_table_id = f"{db}.{yh_ds}.{medication_table_name}"
medication_table = client.get_table(medication_table_id)

comorbidity_table_name = 'dementia_comorbidity_categories_aggregated'
comorbidity_table_id = f"{db}.{yh_ds}.{comorbidity_table_name}"
comorbidity_table = client.get_table(comorbidity_table_id)

comorbidity_before_exposure_table_name = 'dementia_comorbidity_categories_before_exposure_aggregated'
comorbidity_before_exposure_table_id = f"{db}.{yh_ds}.{comorbidity_before_exposure_table_name}"
comorbidity_before_exposure_table = client.get_table(comorbidity_before_exposure_table_id)

# Get the list of column names
comorbidity_column_names = [schema_field.name for schema_field in comorbidity_table.schema]
comorbidity_before_exposure_table_column_names = [schema_field.name for schema_field in comorbidity_before_exposure_table.schema]
medication_column_names = [schema_field.name for schema_field in medication_table.schema]

In [ ]:
columns_list = df_comparison.columns.to_list()

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Feature engineering: Imputation and BMI categorization
df_imputed = df_comparison.copy()

# Convert 'last_bmi_before_exposure' to numeric, handling invalid values
df_imputed['last_bmi_before_exposure'] = pd.to_numeric(
    df_imputed['last_bmi_before_exposure'], errors='coerce'
)

# Impute missing BMI values with the median
median_value = df_imputed['last_bmi_before_exposure'].median()
df_imputed['last_bmi_before_exposure'].fillna(median_value, inplace=True)

# Define BMI bins and labels, including an upper boundary for extreme BMIs
bins = [0, 18.5, 24.9, 29.9, 34.9, 40, float('inf')]
labels = ['Underweight', 'Normal weight', 'Overweight', 'Obese', 'Severely obese', 'Morbidly obese']
category_to_numeric = {
    'Underweight': 1,
    'Normal weight': 2,
    'Overweight': 3,
    'Obese': 4,
    'Severely obese': 5,
    'Morbidly obese': 6
}

# Categorize BMI into ordinal groups
df_imputed['last_bmi_before_exposure_category'] = pd.cut(
    df_imputed['last_bmi_before_exposure'], bins=bins, labels=labels, right=False
)

# Check for uncategorized values
uncategorized_values = df_imputed[df_imputed['last_bmi_before_exposure_category'].isnull()]
print(f"Number of uncategorized rows: {uncategorized_values.shape[0]}")

# Map BMI categories to numeric ordinal values
df_imputed['last_bmi_before_exposure_ordinal'] = df_imputed['last_bmi_before_exposure_category'].map(category_to_numeric)

# Ensure BMI ordinal column is numeric
df_imputed['last_bmi_before_exposure_ordinal'] = df_imputed['last_bmi_before_exposure_ordinal'].astype(float)

# Define covariate columns
other_covariates = ['age_at_diagnosis', 'sex', 'ethnic_group', 'race', 'last_bmi_before_exposure_ordinal', 'mapped_insurance_type']
covariate_columns = health_covariate_columns + other_covariates

# Identify numerical covariates with more than two unique values
numerical_covariates = [
    col for col in df_imputed[covariate_columns].select_dtypes(include=['float64', 'int64']).columns
    if df_imputed[col].nunique() > 2
]

# Identify binary categorical covariates (exactly two unique values)
binary_categorical_covariates = [
    col for col in df_imputed[covariate_columns].columns
    if df_imputed[col].nunique() == 2
]

# Identify non-binary categorical covariates
categorical_covariates = df_imputed[covariate_columns].select_dtypes(include=['object', 'category']).columns.tolist()

# Combine binary and non-binary categorical covariates
all_categorical_covariates = categorical_covariates + binary_categorical_covariates

# Create imputers for numerical and categorical covariates
median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')

# Impute missing values for numerical covariates
df_imputed[numerical_covariates] = median_imputer.fit_transform(df_imputed[numerical_covariates])

# Impute missing values for categorical covariates
df_imputed[all_categorical_covariates] = mode_imputer.fit_transform(df_imputed[all_categorical_covariates])

# Create dummy variables for categorical covariates
df_imputed_with_dummies = pd.get_dummies(
    df_imputed, columns=categorical_covariates, drop_first=True
)

# Identify dummy columns (newly created after one-hot encoding)
dummy_columns = df_imputed_with_dummies.columns.difference(df_imputed.columns)

# Combine binary and dummy columns for covariate analysis
categorical_covariate_columns_with_dummies = (
    list(df_imputed_with_dummies.columns[df_imputed_with_dummies.columns.isin(all_categorical_covariates)]) +
    list(dummy_columns)
)

# Define a threshold for minimum samples in categorical variables
min_samples_threshold = 300
sparse_columns = [
    col for col in categorical_covariate_columns_with_dummies
    if df_imputed_with_dummies[col].value_counts().min() < min_samples_threshold
]

# Retain non-sparse columns for analysis
non_sparse_columns = numerical_covariates + [
    col for col in categorical_covariate_columns_with_dummies if col not in sparse_columns
]

# Drop sparse columns to create the final analysis DataFrame
df_analysis = df_imputed_with_dummies.drop(columns=sparse_columns)
comorbid_columns = [i for i in comorbidity_before_exposure_table_column_names if i.startswith('before_exposure')]
medication_columns = [i for i in medication_column_names if i.startswith('exposure_within_1_year_before_first')]
print (len(comorbid_columns))
print (len(medication_columns))
health_covariate_columns = comorbid_columns + medication_columns

In [ ]:
# Get columns with NaN values and their count
nan_counts = df_analysis.isnull().sum()

# Filter only columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]

# Display the columns and their counts
print("Columns with NaN values and their counts:")
print(columns_with_nan)

#### no limit on follow-up 

In [ ]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter, KaplanMeierFitter
import matplotlib.pyplot as plt
df_analysis = df_analysis[df_analysis['follow_up_post_diagnosis_first_op_exposure'].notnull()]
df_analysis = df_analysis[df_analysis['follow_up_post_diagnosis_first_op_exposure']>=0]
# Prepare data for lifelines
X_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[X_cols].copy()

# Add the 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['follow_up_post_diagnosis_first_op_exposure']
X_lifelines['event'] = df_analysis['deceased_boolean']

# Fit the Cox Proportional Hazards Model using lifelines
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event')

# Get hazard ratios and confidence intervals
hazard_ratios = cph.hazard_ratios_
conf_int = np.exp(cph.confidence_intervals_)

# Print hazard ratios and confidence intervals
print("Hazard Ratios (Exponentiated Coefficients):")
print(hazard_ratios)

print("\n95% Confidence Intervals for Hazard Ratios:")
for var in X_cols:
    hr = hazard_ratios.get(var, np.nan)
    ci_low = conf_int.loc[var, '95% lower-bound'] if var in conf_int.index else np.nan
    ci_high = conf_int.loc[var, '95% upper-bound'] if var in conf_int.index else np.nan
    print(f"{var}: {hr:.2f} ({ci_low:.2f}, {ci_high:.2f})")

# Kaplan-Meier estimator plot
plt.figure(figsize=(10, 6))

# Iterate over unique groups for survival analysis
kmf = KaplanMeierFitter()
for value in df_analysis['exposure_group'].unique():
    mask = df_analysis['exposure_group'] == value
    kmf.fit(durations=df_analysis['follow_up_post_diagnosis_first_op_exposure'][mask],
            event_observed=df_analysis['deceased_boolean'][mask],
            label=f"{value} (n = {mask.sum()})")
    
    # Plot the Kaplan-Meier curve for this group
    kmf.plot_survival_function(ci_show=True)
    
plt.ylim(0, 1)
plt.ylabel(r"Estimated Probability of Survival $\hat{S}(t)$")
plt.xlabel("Time $t$")
plt.legend(loc="best")
plt.title("Kaplan-Meier Survival Curves")
plt.show()

define outcomes  

In [ ]:
# engineer outcomes for survival analysis 
df_analysis['death3000_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 3000), axis=1)
df_analysis['death3000_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 3000), axis=1)
df_analysis['death14_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 14), axis=1)
df_analysis['death14_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 14), axis=1)
df_analysis['death28_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 28), axis=1)
df_analysis['death28_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 28), axis=1)
df_analysis['death30_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 30), axis=1)
df_analysis['death30_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 30), axis=1)
df_analysis['death60_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 60), axis=1)
df_analysis['death60_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 60), axis=1)
df_analysis['death90_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 90), axis=1)
df_analysis['death90_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 90), axis=1)
df_analysis['death180_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 180), axis=1)
df_analysis['death180_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 180), axis=1)
df_analysis['death365_duration'] = df_analysis.apply(lambda row: get_death_duration_column(row['post_onset_post_opioid_death_days'], 365), axis=1)
df_analysis['death365_outcome'] = df_analysis.apply(lambda row: get_death_outcome_column(row['post_onset_post_opioid_death_days'], 365), axis=1)

#### primary analysis : 14 days survival (+ PH assumption test)

In [ ]:

# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for lifelines CoxPHFitter
x_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death14_duration']
X_lifelines['event'] = df_analysis['death14_outcome']

# Verify that columns exist and have no missing values
print(X_lifelines[['duration', 'event']].isnull().sum())

# Fit the Cox Proportional Hazards Model using lifelines
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event')

# Check proportional hazards assumptions
results = cph.check_assumptions(X_lifelines, p_value_threshold=0.05, show_plots=False)

# Extract variables that failed the proportional hazards assumption
from lifelines.statistics import proportional_hazard_test

ph_test_results = proportional_hazard_test(cph, X_lifelines, time_transform="rank")
failed_variables = ph_test_results.summary[
    ph_test_results.summary['p'] < 0.05
].index.tolist()

print(f"Variables that failed the proportional hazards assumption: {failed_variables}")


In [ ]:
# Import the required function
from lifelines.plotting import add_at_risk_counts

# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for lifelines CoxPHFitter
x_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death14_duration']
X_lifelines['event'] = df_analysis['death14_outcome']

# Verify that columns exist and have no missing values
print(X_lifelines[['duration', 'event']].isnull().sum())

# Fit the Cox Proportional Hazards Model using lifelines
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event')

# Get hazard ratios, confidence intervals, and p-values
hazard_ratios = cph.hazard_ratios_
conf_int = np.exp(cph.confidence_intervals_)
p_values = cph.summary['p']  # Extract p-values from the summary

# Function to add stars based on p-value with four levels of significance
def significance_stars(p):
    if p <= 0.0001:
        return '****'  # Very highly statistically significant
    elif p <= 0.001:
        return '***'  # Highly statistically significant
    elif p <= 0.01:
        return '**'  # Statistically significant
    elif p <= 0.05:
        return '*'  # Marginally significant
    else:
        return ''  # Not statistically significant

# Print hazard ratios, confidence intervals, and p-values with stars
print("Hazard Ratios (Exponentiated Coefficients):")
print(hazard_ratios)

print("\n95% Confidence Intervals for Hazard Ratios and Statistical Significance:")
for var in X_lifelines.columns:
    hr = hazard_ratios.get(var, np.nan)
    ci_low = conf_int.loc[var, '95% lower-bound'] if var in conf_int.index else np.nan
    ci_high = conf_int.loc[var, '95% upper-bound'] if var in conf_int.index else np.nan
    p_value = p_values.get(var, np.nan)
    stars = significance_stars(p_value)
    print(f"{var}: {hr:.2f} ({ci_low:.2f}, {ci_high:.2f}), p-value: {p_value:.4f}{stars}")

# Kaplan-Meier estimator plot
fig, ax = plt.subplots(figsize=(10, 6))

# Use lifelines KaplanMeierFitter to plot survival curves
kmf_list = []  # Store each Kaplan-Meier fitter
custom_colors = ['#FF6347', '#4682B4', '#32CD32', '#FFD700']  # Custom colors

for i, value in enumerate(df_analysis['exposure_group'].unique()):
    mask = df_analysis['exposure_group'] == value
    kmf = KaplanMeierFitter()
    kmf.fit(
        durations=df_analysis['death14_duration'][mask],
        event_observed=df_analysis['death14_outcome'][mask],
        label=f"{value} (n = {mask.sum()})"
    )
    kmf.plot_survival_function(ax=ax, ci_show=True, color=custom_colors[i % len(custom_colors)])
    kmf_list.append(kmf)  # Append the fitted Kaplan-Meier object

# Add the number at risk table for all groups
add_at_risk_counts(*kmf_list, ax=ax )

# Finalize the plot
plt.ylim(0.5, 1)
plt.ylabel(r"Estimated Probability of Survival $\hat{S}(t)$", fontsize=14)
plt.xlabel("Time $t$ (days)", fontsize=14)
plt.legend(loc="best", fontsize=12, frameon=False)
plt.title("14-days Survival Curves by Exposure Group", fontsize=16)

# Save the figure as vector images (SVG and PDF)
#plt.savefig("../../figures/main_analysis.svg", format="svg", bbox_inches="tight")
#plt.savefig("../../figures/main_analysis.pdf", format="pdf", bbox_inches="tight")

# Show the plot
plt.show()

# Evaluate the accuracy using Concordance Index (C-Index)
c_index = cph.concordance_index_
print(f"\nConcordance Index (C-Index): {c_index:.3f}")


#### sensitivity analysis adjusted for variables that failed PH assumption test

In [ ]:
# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for lifelines CoxPHFitter
x_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death14_duration']
X_lifelines['event'] = df_analysis['death14_outcome']


# Fit the Cox Proportional Hazards Model using lifelines
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event', strata = ['before_exposure_delirium',
 'before_exposure_hyperlipidemia',
 'before_exposure_schizophrenia_non_mood_psychotic_disorder',
 'exposure_within_1_year_before_first_op_acetaminophen',
 'exposure_within_1_year_before_first_op_benzodiazepines',
 'exposure_within_1_year_before_first_op_insulin',
 'exposure_within_1_year_before_first_op_meperidine',
 'exposure_within_1_year_before_first_op_topical_anesthetic',
 'exposure_within_1_year_before_first_op_warfarin',
 'race_White'])

# Get hazard ratios, confidence intervals, and p-values
hazard_ratios = cph.hazard_ratios_
conf_int = np.exp(cph.confidence_intervals_)
p_values = cph.summary['p']  # Extract p-values from the summary

# Function to add stars based on p-value with four levels of significance
def significance_stars(p):
    if p <= 0.0001:
        return '****'  # Very highly statistically significant
    elif p <= 0.001:
        return '***'  # Highly statistically significant
    elif p <= 0.01:
        return '**'  # Statistically significant
    elif p <= 0.05:
        return '*'  # Marginally significant
    else:
        return ''  # Not statistically significant

# Print hazard ratios, confidence intervals, and p-values with stars
print("Hazard Ratios (Exponentiated Coefficients):")
print(hazard_ratios)

print("\n95% Confidence Intervals for Hazard Ratios and Statistical Significance:")
for var in X_lifelines.columns:
    hr = hazard_ratios.get(var, np.nan)
    ci_low = conf_int.loc[var, '95% lower-bound'] if var in conf_int.index else np.nan
    ci_high = conf_int.loc[var, '95% upper-bound'] if var in conf_int.index else np.nan
    p_value = p_values.get(var, np.nan)
    stars = significance_stars(p_value)
    print(f"{var}: {hr:.2f} ({ci_low:.2f}, {ci_high:.2f}), p-value: {p_value:.4f}{stars}")

# Kaplan-Meier estimator plot
fig, ax = plt.subplots(figsize=(10, 6))

# Use lifelines KaplanMeierFitter to plot survival curves
kmf_list = []  # Store each Kaplan-Meier fitter
custom_colors = ['#FF6347', '#4682B4', '#32CD32', '#FFD700']  # Custom colors

for i, value in enumerate(df_analysis['exposure_group'].unique()):
    mask = df_analysis['exposure_group'] == value
    kmf = KaplanMeierFitter()
    kmf.fit(
        durations=df_analysis['death14_duration'][mask],
        event_observed=df_analysis['death14_outcome'][mask],
        label=f"{value} (n = {mask.sum()})"
    )
    kmf.plot_survival_function(ax=ax, ci_show=True, color=custom_colors[i % len(custom_colors)])
    kmf_list.append(kmf)  # Append the fitted Kaplan-Meier object

# Add the number at risk table for all groups
add_at_risk_counts(*kmf_list, ax=ax, rows_to_show = ['At risk'])

# Set larger font sizes for manuscript publication
plt.rcParams.update({
    'font.size': 16,          # Base font size
    'axes.titlesize': 20,     # Title font size
    'axes.labelsize': 18,     # Axis labels font size
    'xtick.labelsize': 16,    # X-axis tick labels font size
    'ytick.labelsize': 16,    # Y-axis tick labels font size
    'legend.fontsize': 16,    # Legend font size
    'figure.titlesize': 22    # Overall figure title size
})

# Finalize the plot with the updated title
plt.ylim(0.5, 1)
plt.ylabel(r"Estimated Probability of Survival $\hat{S}(t)$", fontsize=18)
plt.xlabel("Time $t$ (days)", fontsize=18)
plt.legend(loc="best", fontsize=16, frameon=False)
plt.title("Kaplan-Meier Survival Curves by Exposure Group Over 14 Days (Adjusted for Variables Violating PH Assumption)", fontsize=20)

# Save the figure as vector images (SVG and PDF)
# plt.savefig("kaplan_meier_adjusted_ph_assumption.svg", format="svg", bbox_inches="tight")
#plt.savefig("../../figures/main_analysis_14days_ph_assumption.pdf", format="pdf", bbox_inches="tight")

# Show the plot
plt.show()


# Evaluate the accuracy using Concordance Index (C-Index)
c_index = cph.concordance_index_
print(f"\nConcordance Index (C-Index): {c_index:.3f}")

#### 60-day survival (to identify 95% confidence interval overlap) 

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'
# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for lifelines CoxPHFitter
x_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death60_duration']
X_lifelines['event'] = df_analysis['death60_outcome']


# Fit the Cox Proportional Hazards Model using lifelines
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event')

# Get hazard ratios, confidence intervals, and p-values
hazard_ratios = cph.hazard_ratios_
conf_int = np.exp(cph.confidence_intervals_)
p_values = cph.summary['p']  # Extract p-values from the summary

# Function to add stars based on p-value with four levels of significance
def significance_stars(p):
    if p <= 0.0001:
        return '****'  # Very highly statistically significant
    elif p <= 0.001:
        return '***'  # Highly statistically significant
    elif p <= 0.01:
        return '**'  # Statistically significant
    elif p <= 0.05:
        return '*'  # Marginally significant
    else:
        return ''  # Not statistically significant

# Print hazard ratios, confidence intervals, and p-values with stars
print("Hazard Ratios (Exponentiated Coefficients):")
print(hazard_ratios)

print("\n95% Confidence Intervals for Hazard Ratios and Statistical Significance:")
for var in X_lifelines.columns:
    hr = hazard_ratios.get(var, np.nan)
    ci_low = conf_int.loc[var, '95% lower-bound'] if var in conf_int.index else np.nan
    ci_high = conf_int.loc[var, '95% upper-bound'] if var in conf_int.index else np.nan
    p_value = p_values.get(var, np.nan)
    stars = significance_stars(p_value)
    print(f"{var}: {hr:.2f} ({ci_low:.2f}, {ci_high:.2f}), p-value: {p_value:.4f}{stars}")
# Kaplan-Meier curves for New Users and Consistent Users
kmf_new_user = KaplanMeierFitter()
kmf_consistent_user = KaplanMeierFitter()

# Fit Kaplan-Meier curve for new users
mask_new_user = df_analysis['exposure_group'] == 'new user'
n_new_user = mask_new_user.sum()  # Sample size for New Users
kmf_new_user.fit(
    durations=df_analysis['death60_duration'][mask_new_user],
    event_observed=df_analysis['death60_outcome'][mask_new_user],
    label=f"New User (n = {n_new_user})"
)

# Fit Kaplan-Meier curve for consistent users
mask_consistent_user = df_analysis['exposure_group'] == 'consistent user'
n_consistent_user = mask_consistent_user.sum()  # Sample size for Consistent Users
kmf_consistent_user.fit(
    durations=df_analysis['death60_duration'][mask_consistent_user],
    event_observed=df_analysis['death60_outcome'][mask_consistent_user],
    label=f"Consistent User (n = {n_consistent_user})"
)

# Plot the survival curves
fig, ax = plt.subplots(figsize=(12, 8))
kmf_new_user.plot(ax=ax, ci_show=True, color='#FF6347')  # Tomato color for New User
kmf_consistent_user.plot(ax=ax, ci_show=True, color='#4682B4')  # SteelBlue color for Consistent User

# Add a manual vertical line at 51 days
ax.axvline(x=51, color='black', linestyle='--', label='Overlap at 51 Days')

# Customize the survival plot
ax.set_ylim(0.5, 1)
ax.set_title('Kaplan-Meier Survival Curves with Overlap at 51 Days', fontsize=16)
ax.set_ylabel('Survival Probability', fontsize=14)
ax.set_xlabel('Time (days)', fontsize=14)
ax.legend(loc='best', fontsize=12)

# Add at-risk counts below the x-axis
max_time = max(kmf_new_user.event_table.index.max(), kmf_consistent_user.event_table.index.max())
time_points = np.arange(0, int(max_time) + 1, step=10)
at_risk_new_user = [kmf_new_user.event_table.loc[t, 'at_risk'] if t in kmf_new_user.event_table.index else 0 for t in time_points]
at_risk_consistent_user = [kmf_consistent_user.event_table.loc[t, 'at_risk'] if t in kmf_consistent_user.event_table.index else 0 for t in time_points]

# Adjust the figure to make room for the at-risk table
fig.subplots_adjust(bottom=0.3)  # Increase the bottom margin

# Add at-risk table
y_position_new_user = -0.15  # Adjust position to below the plot
y_position_consistent_user = -0.20
for i, t in enumerate(time_points):
    ax.text(t, y_position_new_user, f"{at_risk_new_user[i]}", ha='center', fontsize=13, transform=ax.get_xaxis_transform())  # New User at-risk numbers
    ax.text(t, y_position_consistent_user, f"{at_risk_consistent_user[i]}", ha='center', fontsize=13, transform=ax.get_xaxis_transform())  # Consistent User at-risk numbers

# Add labels for at-risk rows
ax.text(-5, y_position_new_user, "At Risk (New Users):", fontsize=13, ha='right', transform=ax.get_xaxis_transform())
ax.text(-5, y_position_consistent_user, "At Risk (Consistent Users):", fontsize=13, ha='right', transform=ax.get_xaxis_transform())

# Save the figure as vector images (optional)
# plt.savefig("kmf_with_overlap_at_51_days.svg", format="svg", bbox_inches="tight")
#plt.savefig("../../figures/main_analysis_60days_overlap.pdf", format="pdf", bbox_inches="tight")

plt.tight_layout()
plt.show()


#### 180-day survival (to characterize the pattern of survival curves) 

In [ ]:
# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for lifelines CoxPHFitter
x_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death180_duration']
X_lifelines['event'] = df_analysis['death180_outcome']


# Fit the Cox Proportional Hazards Model using lifelines
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event')

# Get hazard ratios, confidence intervals, and p-values
hazard_ratios = cph.hazard_ratios_
conf_int = np.exp(cph.confidence_intervals_)
p_values = cph.summary['p']  # Extract p-values from the summary

# Function to add stars based on p-value with four levels of significance
def significance_stars(p):
    if p <= 0.0001:
        return '****'  # Very highly statistically significant
    elif p <= 0.001:
        return '***'  # Highly statistically significant
    elif p <= 0.01:
        return '**'  # Statistically significant
    elif p <= 0.05:
        return '*'  # Marginally significant
    else:
        return ''  # Not statistically significant

# Print hazard ratios, confidence intervals, and p-values with stars
print("Hazard Ratios (Exponentiated Coefficients):")
print(hazard_ratios)

print("\n95% Confidence Intervals for Hazard Ratios and Statistical Significance:")
for var in X_lifelines.columns:
    hr = hazard_ratios.get(var, np.nan)
    ci_low = conf_int.loc[var, '95% lower-bound'] if var in conf_int.index else np.nan
    ci_high = conf_int.loc[var, '95% upper-bound'] if var in conf_int.index else np.nan
    p_value = p_values.get(var, np.nan)
    stars = significance_stars(p_value)
    print(f"{var}: {hr:.2f} ({ci_low:.2f}, {ci_high:.2f}), p-value: {p_value:.4f}{stars}")

# Kaplan-Meier estimator plot
fig, ax = plt.subplots(figsize=(10, 6))

# Use lifelines KaplanMeierFitter to plot survival curves
kmf_list = []  # Store each Kaplan-Meier fitter
custom_colors = ['#FF6347', '#4682B4', '#32CD32', '#FFD700']  # Custom colors

for i, value in enumerate(df_analysis['exposure_group'].unique()):
    mask = df_analysis['exposure_group'] == value
    kmf = KaplanMeierFitter()
    kmf.fit(
        durations=df_analysis['death180_duration'][mask],
        event_observed=df_analysis['death180_outcome'][mask],
        label=f"{value} (n = {mask.sum()})"
    )
    kmf.plot_survival_function(ax=ax, ci_show=True, color=custom_colors[i % len(custom_colors)])
    kmf_list.append(kmf)  # Append the fitted Kaplan-Meier object

# Add the number at risk table for all groups
add_at_risk_counts(*kmf_list, ax=ax, rows_to_show = ['At risk'])

# Set larger font sizes for manuscript publication
plt.rcParams.update({
    'font.size': 16,          # Base font size
    'axes.titlesize': 20,     # Title font size
    'axes.labelsize': 18,     # Axis labels font size
    'xtick.labelsize': 16,    # X-axis tick labels font size
    'ytick.labelsize': 16,    # Y-axis tick labels font size
    'legend.fontsize': 16,    # Legend font size
    'figure.titlesize': 22    # Overall figure title size
})

# Finalize the plot with the updated title
plt.ylim(0.5, 1)
plt.ylabel(r"Estimated Probability of Survival $\hat{S}(t)$", fontsize=18)
plt.xlabel("Time $t$ (days)", fontsize=18)
plt.legend(loc="best", fontsize=16, frameon=False)
plt.title("180 Days Survival Curves by Exposure Group", fontsize=20)

# Save the figure as vector images (SVG and PDF)
# plt.savefig("kaplan_meier_adjusted_ph_assumption.svg", format="svg", bbox_inches="tight")
#plt.savefig("../../figures/main_analysis_180days.pdf", format="pdf", bbox_inches="tight")

# Show the plot
plt.show()


# Evaluate the accuracy using Concordance Index (C-Index)
c_index = cph.concordance_index_
print(f"\nConcordance Index (C-Index): {c_index:.3f}")

#### 60-day Aalen additive model 

In [ ]:
import numpy as np
import pandas as pd
from lifelines import AalenAdditiveFitter
from lifelines.utils import concordance_index
import matplotlib.pyplot as plt

# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for AalenAdditiveFitter
x_cols = ['exposure_float'] + non_sparse_columns
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death60_duration']
X_lifelines['event'] = df_analysis['death60_outcome']

# Fit Aalen's Additive Model using lifelines, focusing on exposure_float
aaf = AalenAdditiveFitter(coef_penalizer=0.2)  # Adjust coef_penalizer for regularization if needed
aaf.fit(X_lifelines, duration_col='duration', event_col='event')

# Get the time-varying coefficient (cumulative hazard) for exposure_float
time_varying_coef_exposure = aaf.cumulative_hazards_['exposure_float']

# Find the time point (date) with the highest hazard coefficient for exposure_float
max_time = time_varying_coef_exposure.idxmax()
max_value = time_varying_coef_exposure.max()

# Print the time point and the highest hazard coefficient
print(f"The time point with the highest hazard coefficient for exposure_float is: {max_time}")
print(f"The highest hazard coefficient for exposure_float is: {max_value}")

# Predict cumulative hazards for each individual (results in a DataFrame with time on the index)
predicted_cumulative_hazards = aaf.predict_cumulative_hazard(X_lifelines)

# Interpolate the cumulative hazard for each individual's duration
hazard_at_duration = X_lifelines.apply(
    lambda row: np.interp(row['duration'], predicted_cumulative_hazards.index, predicted_cumulative_hazards[row.name]), axis=1
)

# Calculate the concordance index using the predicted hazards
ci = concordance_index(X_lifelines['duration'], -hazard_at_duration, X_lifelines['event'])

print(f"Concordance Index: {ci}")

# Plot the time-varying coefficient (cumulative hazard) for exposure_float
plt.figure(figsize=(10, 6))

# Plot time-varying coefficient with Tomato color
plt.plot(time_varying_coef_exposure.index, time_varying_coef_exposure, 
         label="First-time Opioid Exposure", color='tomato', linewidth=2)

# Highlight the time point with the maximum hazard coefficient with Black color
plt.axvline(max_time, color='black', linestyle='--', label=f"Max Hazard at {max_time}", linewidth=1.5)

# Enhance figure aesthetics for publication
plt.xlabel("Time $t$ (days)", fontsize=14)
plt.ylabel("Cumulative Hazard Coefficient", fontsize=14)
plt.title("Time-varying Coefficient for First-time Opioid Exposure in Aalen's Additive Model", fontsize=16)

# Adjust tick parameters for better readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Customize the legend
plt.legend(loc='best', fontsize=12, frameon=False)

# Save the figure as vector images (SVG and PDF)
#plt.savefig("../figures/time_varying_coefficient_exposure.svg", format="svg", bbox_inches="tight")
# plt.savefig("../../figures/time_varying_coefficient_exposure.pdf", format="pdf", bbox_inches="tight")

# Show the plot
plt.show()

#### 14-day survival feature importance test 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from lifelines import CoxPHFitter
from sklearn.preprocessing import MinMaxScaler
from sklearn.inspection import permutation_importance
# Set font to Helvetica globally for the plot
plt.rcParams['font.family'] = 'Helvetica'

# Prepare data for Random Survival Forest and Cox model
x_cols = ['exposure_float'] + non_sparse_columns  # Include exposure_float in the training
X_lifelines = df_analysis[x_cols].copy()

# Add 'duration' and 'event' columns
X_lifelines['duration'] = df_analysis['death14_duration']
X_lifelines['event'] = df_analysis['death14_outcome'].astype(bool)

# Normalize 'age_at_diagnosis' to [0, 1] using MinMaxScaler
scaler = MinMaxScaler()
X_lifelines['age_at_diagnosis'] = scaler.fit_transform(X_lifelines[['age_at_diagnosis']])

# Prepare the survival data format for sksurv
y = Surv.from_dataframe('event', 'duration', X_lifelines)
X = X_lifelines.drop(columns=['duration', 'event'])  # Include exposure_float during training

# Ensure all covariates are numeric
X = X.apply(pd.to_numeric, errors='coerce')

# Drop rows with any missing values
X = X.dropna()

# --- Fit the Random Survival Forest ---
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, random_state=0)
rsf.fit(X, y)

# Compute permutation importance
perm_importance = permutation_importance(rsf, X, y, n_repeats=10, random_state=0)

# --- Fit the Cox Proportional Hazards Model ---
cph = CoxPHFitter()
cph.fit(X_lifelines, duration_col='duration', event_col='event')

# Get the coefficients (log hazard ratios)
coefficients = cph.params_

# Create a DataFrame to display the importance scores, excluding 'exposure_float' from the plot
importance_df = pd.DataFrame({
    'Variable': X.columns,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

# Exclude 'exposure_float' from the plot
importance_df = importance_df[importance_df['Variable'] != 'exposure_float']

# Map the coefficients to the importance DataFrame for color-coding
importance_df['Coefficient'] = importance_df['Variable'].map(coefficients)

# Define colors: red for positive coefficients, blue for negative coefficients
importance_df['Color'] = np.where(importance_df['Coefficient'] > 0, 'red', 'blue')

# Plot the feature importances with color-coding based on the sign of the coefficients
plt.figure(figsize=(10, 6))

# Bar plot with color-coded bars based on the sign of Cox coefficients
plt.barh(importance_df['Variable'].head(10), importance_df['Importance'].head(10), 
         color=importance_df['Color'].head(10))

# Enhancing the plot for publication
plt.xlabel('Permutation Importance', fontsize=14)
plt.title('Top 10 Most Important Features in Random Survival Forest\n(Colored by Cox Coefficients)', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top

# Save the figure as vector images (SVG and PDF)
#plt.savefig("random_survival_forest_importance_renamed.svg", format="svg", bbox_inches="tight")
#plt.savefig("../../figures/random_survival_forest_importance_renamed.pdf", format="pdf", bbox_inches="tight")

# Show the plot
plt.show()